In [15]:
import pandas as pd
import numpy as np
import re

In [ ]:
# upload all files
df1 = pd.read_csv('punks_atributes1.csv')
df2 = pd.read_csv('punks_atributes2.csv')
df3 = pd.read_csv('punks_atributes3.csv')
df4 = pd.read_csv('punks_atributes4.csv')
df5 = pd.read_csv('punks_atributes5.csv')

In [ ]:
# merge into a single csv
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=False)
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
# tenemos 10,000 punks
df.shape

In [ ]:
df.head(10)

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
# some missing values
df[df.traits.isnull()]

In [ ]:
df.fillna('',inplace=True)

In [ ]:
df1 = df.copy()

In [ ]:
def get_number_hidden_traits(text):
    pattern = ',Hidden\s(\w+)'
    try:
        hidden = re.findall(pattern,text)
        return len(hidden)    
    except:
        return 0

In [ ]:
#examples of regex
text1 = 'Blonde Bob,Earring,Green Eye Shadow,Hidden Earring,,Hidden Eear'
text2 = 'Blonde Bob,Earring,Green Eye Shadow'
text3= 'Earring,Hidden bob'

def extract_hidden_traits(text):
    match = re.findall(',Hidden\s(\w+)',text)
    if not match:
        #print("List is empty")
        return '' 
    else:
        return ','.join(match)


extract_hidden_traits(text3)

In [ ]:
df1['total_hidden_traits']=df1['traits'].apply(get_number_hidden_traits)

In [ ]:
df1['hidden_traits']=df1['traits'].apply(extract_hidden_traits)

In [ ]:
text1 = 'Blonde Bob,Earring,Green Eye Shadow,Hidden Earring,Hidden Eear'

def remove_hidden_traits(text):
    match = re.findall(',Hidden\s(\w+)',text)    
    if not match:
        #print("List is empty")
        return text 
    else:
        to_remove= ',Hidden '+',Hidden '.join(match)        
        clean = text.replace(to_remove,'')        
        return clean


remove_hidden_traits(text1)
    

In [ ]:
df1['traits'] = df1['traits'].apply(remove_hidden_traits)

In [ ]:
df1.traits.head()

In [ ]:
df1

In [ ]:
df1.to_csv('raw_punks.csv')

In [ ]:
#get traits 
traits ={}
for items in df1.traits.str.split(','):
    for elem in items:        
        if len(elem) ==0:
            #print(type(elem))
            continue            
        if elem in traits:
            #print('exists, add one')
            traits[elem] += 1
        else:
            #print('not exist, create set to one ')
            traits[elem] = 1
    

In [ ]:
df_traits = pd.DataFrame(list(traits.items()),columns = ['trait','occurences']) 
df_traits.drop('occurences',axis=1,inplace=True)

In [ ]:
df_traits.to_csv('.././to_database/trait.csv')

In [ ]:
hidden_trait = []
for row in df1.hidden_traits.str.split(','):
    for elem in row:
        if len(elem) ==0:
            continue
        else: 
            hidden_trait.append(elem)


In [ ]:
hidden_trait= list(set(hidden_trait))

In [ ]:
df_hidden_trait = pd.DataFrame(hidden_trait,columns=['name'])

In [ ]:
df_hidden_trait.to_csv(r'.././to_database/hidden_trait.csv',index=True)

In [ ]:
df2=df1.copy()

In [ ]:
df2.head()

In [ ]:
tipos = {'Alien':1,'Ape':2,'Zombie':3,'Male':4,'Female':5}

In [ ]:
df2['type_id']= df2['types'].map(tipos)

In [ ]:
piel = {'Alien':1,'Ape':2,'Zombie':3,'Albino':4,'Dark':5,'Light':6,'Mid':7}

In [ ]:
df2['skin_id']= df2['skin'].map(piel)

In [ ]:
df2.head()

In [ ]:
df2.tail()

In [ ]:
df2.set_index('id',inplace=True)

In [ ]:
df2

In [ ]:
df2[['type_id','rarity','skin_id','total_traits','total_hidden_traits']].to_csv(r'.././to_database/punk.csv',index=True)

In [ ]:
df2

In [ ]:
df_trait = pd.read_csv('.././to_database/trait.csv')

In [ ]:
df_trait.set_index('id',inplace=True)

In [ ]:
df_trait.head()

In [ ]:
df2.head()

In [ ]:
def get_trait_id(df,col_name,word):
    index = df.index[df[col_name]==word].values[0]
    #print(index)
    return index

In [ ]:
punk_id= []
trait_id = []
for item in df2.traits.iteritems():    
    words = item[1].split(',') 
    #print(words)
    for word in words:
        if len(word) == 0:
            continue
        else:   
            punk_id.append(item[0])
            trait_id.append(get_trait_id(df_trait,'trait',word))

In [ ]:
df_contains_trait = pd.DataFrame({'punk_id':punk_id,'trait_id':trait_id})

In [ ]:
df_contains_trait.to_csv(r'.././to_database/contains_trait.csv',index=False)

In [ ]:
df_contains_trait

In [ ]:
df_hidden_trait = pd.read_csv('.././to_database/hidden_trait.csv')
df_hidden_trait.set_index('id',inplace=True)
punk_id= []
trait_id = []
for item in df2.hidden_traits.iteritems():
    words = item[1].split(',')    
    #print(words)
    for word in words:        
        if len(word) == 0:
            continue
        else:
            punk_id.append(item[0])
            trait_id.append(get_trait_id(df_hidden_trait,'name',word))

            
df_contains_hidden_trait = pd.DataFrame({'punk_id':punk_id,'hidden_trait_id':trait_id})

In [ ]:
df_contains_hidden_trait.to_csv(r'.././to_database/contains_hidden_trait.csv',index=False)

In [ ]:
df_contains_hidden_trait